In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Veriyi CSV dosyasından oku
df = pd.read_csv("heart.csv")

# İlk 5 satırı göster
print("İlk 5 Satır:")
print(df.head())

# Veri hakkında genel bilgiler
print("\nVeri Bilgileri:")
print(df.info())

# Sayısal özelliklerin istatistikleri
print("\nİstatistikler:")
print(df.describe())

İlk 5 Satır:
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   52    1   0       125   212    0        1      168      0      1.0      2   
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  target  
0   2     3       0  
1   0     3       0  
2   0     3       0  
3   1     3       0  
4   3     2       0  

Veri Bilgileri:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol 

In [3]:
# Kategorik sütunları belirle (veri setine göre)
categorical_columns = ['cp', 'restecg', 'slope', 'ca', 'thal']

# One-Hot Encoding uygula
df_encoded = pd.get_dummies(df, columns=categorical_columns)

# İlk 3 satırı göster
print("One-Hot Encoding Sonrası Veri:")
print(df_encoded.head(3))

One-Hot Encoding Sonrası Veri:
   age  sex  trestbps  chol  fbs  thalach  exang  oldpeak  target  cp_0  ...  \
0   52    1       125   212    0      168      0      1.0       0  True  ...   
1   53    1       140   203    1      155      1      3.1       0  True  ...   
2   70    1       145   174    0      125      1      2.6       0  True  ...   

   slope_2   ca_0   ca_1   ca_2   ca_3   ca_4  thal_0  thal_1  thal_2  thal_3  
0     True  False  False   True  False  False   False   False   False    True  
1    False   True  False  False  False  False   False   False   False    True  
2    False   True  False  False  False  False   False   False   False    True  

[3 rows x 28 columns]


In [4]:
# Normalleştirilecek sayısal sütunlar
numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# StandardScaler ile normalleştirme
scaler = StandardScaler()
df_encoded[numerical_features] = scaler.fit_transform(df_encoded[numerical_features])

# Normalleştirilmiş veriyi göster
print("\nNormalleştirilmiş Veri:")
print(df_encoded.head(3))


Normalleştirilmiş Veri:
        age  sex  trestbps      chol  fbs   thalach  exang   oldpeak  target  \
0 -0.268437    1 -0.377636 -0.659332    0  0.821321      0 -0.060888       0   
1 -0.158157    1  0.479107 -0.833861    1  0.255968      1  1.727137       0   
2  1.716595    1  0.764688 -1.396233    0 -1.048692      1  1.301417       0   

   cp_0  ...  slope_2   ca_0   ca_1   ca_2   ca_3   ca_4  thal_0  thal_1  \
0  True  ...     True  False  False   True  False  False   False   False   
1  True  ...    False   True  False  False  False  False   False   False   
2  True  ...    False   True  False  False  False  False   False   False   

   thal_2  thal_3  
0   False    True  
1   False    True  
2   False    True  

[3 rows x 28 columns]


In [5]:
# Hedef değişkeni (target) ve özellikleri ayır
X = df_encoded.drop('target', axis=1)  # Özellikler
y = df_encoded['target']               # Hedef

# Veriyi %80 eğitim, %20 test olarak böl
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Boyutları kontrol et
print("Eğitim verisi boyutu:", X_train.shape)
print("Test verisi boyutu:", X_test.shape)

Eğitim verisi boyutu: (820, 27)
Test verisi boyutu: (205, 27)


In [6]:
pip install xgboost joblib

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
    --------------------------------------- 2.1/150.0 MB 7.9 MB/s eta 0:00:19
    --------------------------------------- 3.7/150.0 MB 9.9 MB/s eta 0:00:15
   - -------------------------------------- 6.3/150.0 MB 9.1 MB/s eta 0:00:16
   -- ------------------------------------- 8.7/150.0 MB 9.6 MB/s eta 0:00:15
   --- ------------------------------------ 11.8/150.0 MB 10.5 MB/s eta 0:00:14
   --- ------------------------------------ 14.7/150.0 MB 11.1 MB/s eta 0:00:13
   ---- ----------------------------------- 17.8/150.0 MB 11.5 MB/s eta 0:00:12
   ----- ---------------------------------- 20.7/150.0 MB 11.8 MB/s eta 0:00:11
   ------ --------------------------------- 23.6/150.0 MB 12.0 MB/s eta 0:00:11
   ------- -------------------------------- 26.5/150.0 MB 12.2 MB/s eta 0:00:11
   ------- -------------------------------- 29.6/150.0 MB 12.4 MB/

In [7]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV

In [8]:
# Modeli tanımla (XGBoost sınıflandırıcı)
model = xgb.XGBClassifier()

# Hiperparametre grid'i
param_grid = {
    'max_depth': [3, 5, 7],          # Ağacın maksimum derinliği
    'learning_rate': [0.01, 0.1],     # Öğrenme hızı
    'n_estimators': [100, 200]        # Ağaç sayısı
}

# GridSearchCV ile en iyi parametreleri bul
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,                             # 5 kat çapraz doğrulama
    scoring='f1',                     # F1-Score'a göre optimizasyon
    verbose=1                         # İşlem detaylarını göster
)

# Eğitim verisi ile modeli eğit
grid_search.fit(X_train, y_train)

# En iyi modeli seç
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [9]:
print("En İyi Parametreler:", grid_search.best_params_)

En İyi Parametreler: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200}


In [10]:
y_pred = best_model.predict(X_test)

In [11]:
print("Doğruluk (Accuracy):", accuracy_score(y_test, y_pred))
print("F1-Skor (F1-Score):", f1_score(y_test, y_pred))

Doğruluk (Accuracy): 0.9853658536585366
F1-Skor (F1-Score): 0.9852216748768473


In [12]:
import joblib

# Modeli kaydet
joblib.dump(best_model, 'heart_disease_model.pkl')

# Kaydedilen dosyayı kontrol et
import os
print("Kaydedilen Dosya:", os.listdir())

Kaydedilen Dosya: ['.ipynb_checkpoints', 'heart.csv', 'heart.ipynb', 'heart_disease_model.pkl']
